# 프로젝트: 퍼스널 컬러 진단 - CNN

## 데이터
1. 유명인 웜톤 100장, 쿨톤 100장  
2. 일반인 웜톤 490장, 쿨톤 186장  

## 목표: 간단한 CNN 분류

## (1) 라이브러리 및 데이터 불러오기

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

import numpy as np
import cv2
import os
import glob
from PIL import Image

In [2]:
# 파일 불러오는 함수
def load_images_from_directory(directory_path):
    image_paths = glob.glob(directory_path + '/*.jpg')  # 디렉토리 내의 모든 jpg 파일 경로 찾기
    image_paths.extend(glob.glob(directory_path + '/*.jpeg'))  # 디렉토리 내의 모든 jpeg 파일 경로 추가
    image_paths.extend(glob.glob(directory_path + '/*.JPG'))  # 디렉토리 내의 모든 JPG 파일 경로 추가

    images = []
    i = 0
    for image_path in image_paths:
        if i == 150:
            break
        image = Image.open(image_path)
        image_np = np.array(image)
        images.append(image_np)
        i += 1

    return images

In [3]:
# 모든 이미지 불러오기
directory_path = "/aiffel/aiffel/project/first-repository/data/four_seasons/train_warm_1"  # 디렉토리 경로
images = load_images_from_directory(directory_path)

In [ ]:
directory_path2 = "/aiffel/aiffel/project/first-repository/data/four_seasons/train_cool_1/cool_1"  # 디렉토리 경로
images2 = load_images_from_directory(directory_path2)

In [ ]:
length = len(images)
print(length)
print(len(images2))

# 이미지 리스트 합치기
images = np.concatenate((images, images2), axis=0)
print(len(images))

## (2) 데이터 전처리

In [ ]:
def preprocess_image(image):
    # 이미지 크기 조절
    image = tf.image.resize(image, (224, 224))
    # 이미지를 [0, 1] 범위로 정규화
    image = image / 255.0
    return image

images = np.array([preprocess_image(image) for image in images])
print(images.shape)

- KNN과 다르게 평탄화 작업을 제거하였고 리사이즈 함수가 다르다

In [ ]:
# labels에 대한 코드
num = len(images)
labels = np.zeros(num)

# 뒤쪽의 50개의 요소를 1로 변경
labels[length:] = 1

# 0이 웜톤 1이 쿨톤
print(len(images))
print(labels.shape)
print(labels)

## (3) train, validation 분리하기

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(images,
                                                  labels,
                                                  test_size=0.3,
                                                  random_state=55)

print("x_train: ", x_train.shape)
print("y_train: ", y_train.shape)
print("x_val: ", x_val.shape)
print("y_val: ", y_val.shape)

In [ ]:
# 데이터를 텐서로 변환
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
x_val = tf.convert_to_tensor(x_val, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.int32)
y_val = tf.convert_to_tensor(y_val, dtype=tf.int32)

## (4) 모델 정의 및 컴파일

In [ ]:
# 모델 정의
model = models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')  # 두 개의 클래스로 분류
])

# 모델 컴파일
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


## (5) 모델 학습하기

In [ ]:
# 모델 학습
num_epochs = 20
model.fit(x_train, y_train, epochs=num_epochs, batch_size=64)

print("Training finished!")

## (6) 예측하기

In [ ]:
pred = model.predict(x_val)
pred_class = np.argmax(pred, axis=1)
y_val = np.array(y_val)

In [ ]:
print(pred_class)
print(y_val)

In [ ]:
acc = np.mean(pred_class == y_val)
print('accuracy: %f' % (acc,))

ramdom_state를 변화시키면서 train, val 나누는 코드부터 반복 시도함: 200장  
1차 시도: 0.87(42)  
2차 시도: 0.95(1004)  
3차 시도: 0.85(55)  
4차 시도: 0.88(33)  
5차 시도: 0.83(316)  
ramdom_state를 변화시키면서 train, val 나누는 코드부터 반복 시도함: 250장  
1차 시도: 0.87(316)  
2차 시도: 0.95(55)  
3차 시도: 0.88(33)  
4차 시도: 0.87(42)  
5차 시도: 0.89(1004)  
ramdom_state를 변화시키면서 train, val 나누는 코드부터 반복 시도함: 300장  
1차 시도: 0.90(1004)    
2차 시도: 0.90(33)  
-이때쯤 커널 죽음-  

## (7) TEST

In [ ]:
class_name = ["웜톤", "쿨톤"]

In [ ]:
# test 불러오기 (낱개 버전)
test_path = "/aiffel/aiffel/project/first-repository/data/four_seasons/TEST/박은빈.jpg"
x_test = Image.open(test_path)
x_test = np.array(x_test)
print(x_test.shape)

# 배열의 크기 변환
x_test = x_test.reshape((1, x_test.shape[0], x_test.shape[1], 3))
print("변환된 배열 크기:", x_test.shape)

# 전처리(resize, 정규화)
x_test = preprocess_image(x_test)
print(x_test.shape)

# 만약 앱구현과 연결된다면 저장된 모델을 다시 돌릴 필요없이 저장된 최적의 모델을 사용하도록 수정해야함
# test
pred = model.predict(x_test)
pred_class = np.argmax(pred, axis=1)
print("당신은", class_name[int(pred_class)], "입니다~!")
print("웜톤일 확률: ", np.round(pred[:, 0],3))
print("쿨톤일 확률: ", np.round(pred[:, 1],3))

In [ ]:
# 박은빈 겨울쿨이므로 쿨톤 맞춤!!